In [202]:
import pandas 
df = pandas.read_csv("Chocolate Sales (2).csv")

In [203]:
df.head()


,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,"$5,320.00",180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,"$7,896.00",94
2,Gigi Bohling,India,Peanut Butter Cubes,07/07/2022,"$4,501.00",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27/04/2022,"$12,726.00",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24/02/2022,"$13,685.00",184


In [204]:
df.isna().sum()

Sales Person     0
Country          0
Product          0
Date             0
Amount           0
Boxes Shipped    0
dtype: int64

In [205]:
df.duplicated().sum()

np.int64(0)

In [206]:
df.describe

<bound method NDFrame.describe of           Sales Person    Country              Product        Date  \
0       Jehu Rudeforth         UK      Mint Chip Choco  04/01/2022   
1          Van Tuxwell      India        85% Dark Bars  01/08/2022   
2         Gigi Bohling      India  Peanut Butter Cubes  07/07/2022   
3         Jan Morforth  Australia  Peanut Butter Cubes  27/04/2022   
4       Jehu Rudeforth         UK  Peanut Butter Cubes  24/02/2022   
...                ...        ...                  ...         ...   
3277  Karlen McCaffrey  Australia  Spicy Special Slims  17/05/2024   
3278    Jehu Rudeforth        USA           White Choc  07/06/2024   
3279      Ches Bonnell     Canada  Organic Choco Syrup  26/07/2024   
3280    Dotty Strutley      India              Eclairs  28/07/2024   
3281  Karlen McCaffrey      India       70% Dark Bites  23/05/2024   

          Amount  Boxes Shipped  
0      $5,320.00            180  
1      $7,896.00             94  
2      $4,501.00       

In [207]:
'''
HERE, WE HAVE DATES IN OUR DATASET, SO WE ARE PERFORMING THE FOLLOWING CHANGES (NOTE: FOR EVERY DATASET IN WHICH DATES ARE PRESENT WE HAVE TO FOLLOW THIS)
'''
df['Date'] = pandas.to_datetime(df['Date'], dayfirst=True)

#  Extracting numerical features from the date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Droping the original Date column (it's no longer needed, because we have transformed it)
df = df.drop('Date', axis=1)

In [208]:
'''
WE HAD TO REMOVE DOLLAR SIGN, BECAUSE IT WAS PREVENTING US TO CONVERT IT INTO FLOAT
'''

df['Amount'] = df['Amount'].replace(r'[\$,]', '', regex=True).astype(float)

In [209]:
'''
HERE I HAVE DECIDED TO PREDICT AMOUNT
'''
#droped the sales person colomn becasuse it was useless
X = df.drop(['Amount','Sales Person'], axis=1)
y = df['Amount']


In [210]:
print(X)

        Country              Product  Boxes Shipped  Year  Month  Day  \
0            UK      Mint Chip Choco            180  2022      1    4   
1         India        85% Dark Bars             94  2022      8    1   
2         India  Peanut Butter Cubes             91  2022      7    7   
3     Australia  Peanut Butter Cubes            342  2022      4   27   
4            UK  Peanut Butter Cubes            184  2022      2   24   
...         ...                  ...            ...   ...    ...  ...   
3277  Australia  Spicy Special Slims            354  2024      5   17   
3278        USA           White Choc            121  2024      6    7   
3279     Canada  Organic Choco Syrup            238  2024      7   26   
3280      India              Eclairs            397  2024      7   28   
3281      India       70% Dark Bites            355  2024      5   23   

      DayOfWeek  
0             1  
1             0  
2             3  
3             2  
4             3  
...         ...

In [211]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [212]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [213]:
for col in X_train.select_dtypes(['object']).columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [214]:
from xgboost import XGBRegressor
xgbreg = XGBRegressor(random_state=42,enable_categorical=True)
xgbreg.fit(X_train,y_train)
y_pred = xgbreg.predict(X_test)

In [215]:
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)


0.6506160256068654
1814.8481525915831
2424.765665938839


In [216]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 500, 1000], 
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
}

grid_search = GridSearchCV(
    estimator=XGBRegressor(enable_categorical=True, random_state=42),
    param_grid=param_grid, 
    cv=3, 
    scoring='r2'
)

grid_search.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBRegressor(...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], 'n_estimators': [100, 500, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;

In [217]:
print(f"Best Score: {grid_search.best_score_}")
print(f"Best Settings: {grid_search.best_params_}")

Best Score: 0.4502656761088318
Best Settings: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000}


In [218]:
'''
HERE USING THE best_estimator TO GET THE BEST SCORE
NOTE: I AM ONLY DOING THIS BECASUE AS YOU CAN SEE ABOVE THE DEFAULT PERFORMED BETTER THAN THE MEAN OF GRIDSEARCH, SO I AM JUST CHECKING THAT
'''
best_model = grid_search.best_estimator_

y_pred_best = best_model.predict(X_test)

print("Tuned Model R2 Score:", r2_score(y_test, y_pred_best))

Tuned Model R2 Score: 0.6883274258269156


In [219]:
from scipy.stats import uniform, randint

param_grid_search = {
    'n_estimators': randint(100, 1000), 
    'learning_rate': uniform(0.01, 0.1),
    'max_depth':randint(3,10)
}

random_search = RandomizedSearchCV(
    estimator=XGBRegressor(enable_categorical=True, random_state=42),
    param_distributions=param_grid_search,       
    cv=3,
    n_iter=27,
    scoring='r2',
    random_state=42,   
    n_jobs=-1           # parallel processing
)

random_search.fit(X_train, y_train)

print(f"Best Score: {random_search.best_score_}")
print(f"Best Settings: {random_search.best_params_}")

Best Score: 0.4501564636053285
Best Settings: {'learning_rate': np.float64(0.04824619912671628), 'max_depth': 6, 'n_estimators': 931}


In [220]:
best_model = random_search.best_estimator_

y_pred_best = best_model.predict(X_test)

print("Tuned Model R2 Score:", r2_score(y_test, y_pred_best))

Tuned Model R2 Score: 0.6596378863848213


In [ ]:
'''
NOW TRYING TO PREDICT COUNTRY , FOR PRACTICE
'''

In [231]:
a = df.drop(['Country','Sales Person'], axis=1)
b = df['Country']

In [240]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

for col in a.select_dtypes(['object']).columns:
    a[col] = a[col].astype('category')

le = LabelEncoder()
b_encoded = le.fit_transform(b) 

a_train, a_test, b_train_encoded, b_test_encoded = train_test_split(a, b_encoded, test_size=0.2, random_state=42)

xgbclass = XGBClassifier(
    tree_method="hist",        # for categorical data
    enable_categorical=True, 
    random_state=42
)

xgbclass.fit(a_train, b_train_encoded)
y_pred_encoded = xgbclass.predict(a_test)

# Convert numbers back to Country names for the report
y_pred_names = le.inverse_transform(y_pred_encoded)
y_test_names = le.inverse_transform(b_test_encoded)

print("\nClassification Report:\n", classification_report(y_test_names, y_pred_names))


Classification Report:
               precision    recall  f1-score   support

   Australia       0.85      0.85      0.85       123
      Canada       0.90      0.90      0.90        92
       India       0.86      0.96      0.91       104
 New Zealand       0.93      0.90      0.91       101
          UK       0.90      0.88      0.89       113
         USA       0.92      0.89      0.91       124

    accuracy                           0.89       657
   macro avg       0.89      0.90      0.89       657
weighted avg       0.89      0.89      0.89       657



In [254]:
parameters = {
    'n_estimators': [100, 500, 1000], 
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
}

gridsearch = GridSearchCV(
    estimator=XGBClassifier(enable_categorical=True, random_state=42),
    param_grid=parameters, 
    cv=3, 
    scoring='accuracy'
)

gridsearch.fit(a_train, b_train_encoded)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], 'n_estimators': [100, 500, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is disp

In [255]:
print(f"Best Score: {gridsearch.best_score_}")
print(f"Best Settings: {gridsearch.best_params_}")

Best Score: 0.7360000000000001
Best Settings: {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 1000}


In [250]:
bestmodel = gridsearch.best_estimator_

y_predbest = bestmodel.predict(a_test)

print(classification_report(b_test_encoded, y_predbest))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       123
           1       0.97      0.93      0.95        92
           2       0.90      0.96      0.93       104
           3       0.95      0.93      0.94       101
           4       0.88      0.88      0.88       113
           5       0.93      0.91      0.92       124

    accuracy                           0.92       657
   macro avg       0.92      0.92      0.92       657
weighted avg       0.92      0.92      0.92       657



In [251]:
param_gridsearch = {
    'n_estimators': randint(100, 1000), 
    'learning_rate': uniform(0.01, 0.1),
    'max_depth':randint(3,10)
}

randomsearch = RandomizedSearchCV(
    estimator=XGBClassifier(enable_categorical=True, random_state=42),
    param_distributions=param_gridsearch,       
    cv=3,
    n_iter=27,
    scoring='accuracy',
    random_state=42,   
    n_jobs=-1           # parallel processing
)

randomsearch.fit(a_train, b_train_encoded)

print(f"Best Score: {randomsearch.best_score_}")
print(f"Best Settings: {randomsearch.best_params_}")

Best Score: 0.7390476190476191
Best Settings: {'learning_rate': np.float64(0.08272719958564209), 'max_depth': 9, 'n_estimators': 555}


In [253]:
bestmodel = randomsearch.best_estimator_

y_pred_best = bestmodel.predict(a_test)

print(classification_report(b_test_encoded, y_predbest))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       123
           1       0.97      0.93      0.95        92
           2       0.90      0.96      0.93       104
           3       0.95      0.93      0.94       101
           4       0.88      0.88      0.88       113
           5       0.93      0.91      0.92       124

    accuracy                           0.92       657
   macro avg       0.92      0.92      0.92       657
weighted avg       0.92      0.92      0.92       657

